With the OpenVINO Toolkit fundamentals down, you're ready to move onto more topics to get your edge app up and running. Learn about handling input streams, MQTT and more as you finish the tutorial!

## Introduction

[Youtube Video](https://youtu.be/IBP4tsdFRPg)

## OpenCV Basics

[Youtube Video](https://youtu.be/c-pyJ3XwWko)

We can check on [OpenCV Tutorials](https://docs.opencv.org/master/d9/df8/tutorial_root.html).

## Handling Input Streams

[Youtube Video](https://youtu.be/F-7ofR4pdNE)

## Exercise: Handling Input Streams

We implement a function that can handle camera, video or webcam data as input.

## Solution: Handling Input Streams

[Youtube Video](https://youtu.be/p6Hd3dnf-LY)

**Note:** There are two small changes from the code on video for running on Linux machines versus Mac.

- On Mac, `cv2.VideoWriter` uses `cv2.VideoWriter_fourcc('M','J','P','G')` to write an `.mp4` file, while Linux uses `0x00000021`.
- On Mac, the output with the given code on using Canny Edge Detection will run fine. However, on Linux, you'll need to use `np.dstack` to make a 3-channel array to write back to the `out` file, or else the video won't be able to be opened correctly: `frame = np.dstack((frame, frame, frame))`

We use `cv2.VideoCapture()` and open the capture stream:

In [ ]:
capture = cv2.VideoCapture(input_stream)
capture.open(args.input)

while capture.isOpened():
    flag, frame = cap.read()
    if not flag:
        break

It's a bit outside of the instructions, but it's also important to check whether a key gets pressed within the while loop, to make it easier to exit.

We can use:

In [ ]:
# to check for a keypress
key_pressed = cv2.waitKey(60)

# and then to break the loop, if needed. Key 27 is the Escape button.

Re-size the frame to 100x100

In [ ]:
image = cv2.resize(frame, (100, 100))

In [ ]:
edges = cv2.Canny(image,100,200)

# Display the resulting frame if it's video, or save it if it is an image

For video

In [ ]:
cv2.imshow('display', edges)

For an image

In [ ]:
cv2.imwrite('output.jpg', edges)

In [ ]:
# Close the stream and any windows at the end of the application
capture.release()
cv2.destroyAllWindows()

#### Testing the Implementation

In [ ]:
!python app.py -i blue-car.jpg

In [ ]:
!python app.py -i test_video.mp4

## Gathering Useful Information from Model Outputs

[Youtube Video](https://youtu.be/uNoIZI9bm6U)

## Exercise: Process Model Outputs

We use video input and process model outputs.

#### app.py

In [ ]:
import argparse
import cv2
from inference import Network

INPUT_STREAM = "pets.mp4"
CPU_EXTENSION = "/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so"

def get_args():
    '''
    Gets the arguments from the command line.
    '''
    parser = argparse.ArgumentParser("Run inference on an input video")
    # -- Create the descriptions for the commands
    m_desc = "The location of the model XML file"
    i_desc = "The location of the input file"
    d_desc = "The device name, if not 'CPU'"

    # -- Add required and optional groups
    parser._action_groups.pop()
    required = parser.add_argument_group('required arguments')
    optional = parser.add_argument_group('optional arguments')

    # -- Create the arguments
    required.add_argument("-m", help=m_desc, required=True)
    optional.add_argument("-i", help=i_desc, default=INPUT_STREAM)
    optional.add_argument("-d", help=d_desc, default='CPU')
    args = parser.parse_args()

    return args

def assess_scene(result, counter, incident_flag):
    '''
    Based on the determined situation, potentially send
    a message to the pets to break it up.
    '''
    if result[0][1] == 1 and not incident_flag:
        timestamp = counter / 30
        print("Log: Incident at {:.2f} seconds.".format(timestamp))
        print("Break it up!")
        incident_flag = True
    elif result[0][1] != 1:
        incident_flag = False
    
    return incident_flag

def infer_on_video(args):
    # Initialize the Inference Engine
    plugin = Network()

    # Load the network model into the IE
    plugin.load_model(args.m, args.d, CPU_EXTENSION)
    net_input_shape = plugin.get_input_shape()

    # Get and open video capture
    cap = cv2.VideoCapture(args.i)
    cap.open(args.i)

    # Process frames until the video ends, or process is exited
    counter = 0
    incident_flag = False
    while cap.isOpened():
        # Read the next frame
        flag, frame = cap.read()
        if not flag:
            break
        key_pressed = cv2.waitKey(60)
        counter += 1

        # Pre-process the frame
        p_frame = cv2.resize(frame, (net_input_shape[3], net_input_shape[2]))
        p_frame = p_frame.transpose((2,0,1))
        p_frame = p_frame.reshape(1, *p_frame.shape)

        # Perform inference on the frame
        plugin.async_inference(p_frame)

        # Get the output of inference
        if plugin.wait() == 0:
            result = plugin.extract_output()
            ### TODO: Process the output
            incident_flag = assess_scene(result, counter, incident_flag)
            
        # Break if escape key pressed
        if key_pressed == 27:
            break

    # Release the capture and destroy any OpenCV windows
    cap.release()
    cv2.destroyAllWindows()


def main():
    args = get_args()
    infer_on_video(args)


if __name__ == "__main__":
    main()


## Solution: Process Model Outputs

[Youtube Video](https://youtu.be/s35d7IvQliE)

Before the video loop

In [ ]:
counter = 0
incident_flag = False

In [ ]:
def assess_scene(result, counter, incident_flag):
    '''
    Based on the determined situation, potentially send
    a message to the pets to break it up.
    '''
    if result[0][1] == 1 and not incident_flag:
        timestamp = counter / 30
        print("Log: Incident at {:.2f} seconds.".format(timestamp))
        print("Break it up!")
        incident_flag = True
    elif result[0][1] != 1:
        incident_flag = False

    return incident_flag

And we call the function the loop right after the result is available:

In [ ]:
incident_flag = assess_scene(result, counter, incident_flag)

#### Running the App

In [ ]:
!python app.py -m model.xml

## Intro to MQTT

[Youtube Video](https://youtu.be/xe8137UtNCQ)

We can review:
- [Main site for MQTT](http://mqtt.org/)
- [A helpful post about MQTT basics](https://internetofthingsagenda.techtarget.com/definition/MQTT-MQ-Telemetry-Transport)

## Communicating with MQTT

[Youtube Video](https://youtu.be/3Jh-akmlmmU)

We can review:
- [`paho-mqtt` Python library documentation](https://pypi.org/project/paho-mqtt/)

## Streaming Images to a Server

[Youtube Video](https://youtu.be/MyutnkUqIv4)

We can review:
- [FFMPEG](https://www.ffmpeg.org/)
- [Set up Your Own Server on Linux](https://opensource.com/article/19/1/basic-live-video-streaming-server)
- [Use Flask and Python for Streaming](https://www.pyimagesearch.com/2019/09/02/opencv-stream-video-to-web-browser-html-page/)

## Handling Statistics and Images from a Node Server

[Youtube Video](https://youtu.be/qHQ9HjmRGWw)

We can review:
- [Node.js](https://nodejs.org/en/about/)
- [Front End Developer Nanodegree program](https://www.udacity.com/course/front-end-web-developer-nanodegree--nd0011)

## Exercise: Server Communications

We use MQTT and FFMPEG in this exercise.

#### app.py

In [ ]:
import argparse
import cv2
import sys
import numpy as np
import socket
import json
import paho.mqtt.client as mqtt
from random import randint
from inference import Network
### TODO: Import any libraries for MQTT and FFmpeg

INPUT_STREAM = "test_video.mp4"
CPU_EXTENSION = "/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so"
ADAS_MODEL = "/home/workspace/models/semantic-segmentation-adas-0001.xml"


CLASSES = ['road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 
'traffic_light', 'traffic_sign', 'vegetation', 'terrain', 'sky', 'person',
'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'ego-vehicle']

# MQTT server environment variables
HOSTNAME = socket.gethostname()
IPADDRESS = socket.gethostbyname(HOSTNAME)
MQTT_HOST = IPADDRESS
MQTT_PORT = 3004 ### TODO: Set the Port for MQTT
MQTT_KEEPALIVE_INTERVAL = 60

def get_args():
    '''
    Gets the arguments from the command line.
    '''
    parser = argparse.ArgumentParser("Run inference on an input video")
    # -- Create the descriptions for the commands
    i_desc = "The location of the input file"
    d_desc = "The device name, if not 'CPU'"

    # -- Create the arguments
    parser.add_argument("-i", help=i_desc, default=INPUT_STREAM)
    parser.add_argument("-d", help=d_desc, default='CPU')
    args = parser.parse_args()

    return args


def draw_masks(result, width, height):
    '''
    Draw semantic mask classes onto the frame.
    '''
    # Create a mask with color by class
    classes = cv2.resize(result[0].transpose((1,2,0)), (width,height), 
        interpolation=cv2.INTER_NEAREST)
    unique_classes = np.unique(classes)
    out_mask = classes * (255/20)
    
    # Stack the mask so FFmpeg understands it
    out_mask = np.dstack((out_mask, out_mask, out_mask))
    out_mask = np.uint8(out_mask)

    return out_mask, unique_classes


def get_class_names(class_nums):
    class_names= []
    for i in class_nums:
        class_names.append(CLASSES[int(i)])
    return class_names


def infer_on_video(args, model):
    ### TODO: Connect to the MQTT server
    client = mqtt.Client()
    client.connect(MQTT_HOST, MQTT_PORT,
                  MQTT_KEEPALIVE_INTERVAL)

    # Initialize the Inference Engine
    plugin = Network()

    # Load the network model into the IE
    plugin.load_model(model, args.d, CPU_EXTENSION)
    net_input_shape = plugin.get_input_shape()

    # Get and open video capture
    cap = cv2.VideoCapture(args.i)
    cap.open(args.i)

    # Grab the shape of the input 
    width = int(cap.get(3))
    height = int(cap.get(4))

    # Process frames until the video ends, or process is exited
    while cap.isOpened():
        # Read the next frame
        flag, frame = cap.read()
        if not flag:
            break
        key_pressed = cv2.waitKey(60)

        # Pre-process the frame
        p_frame = cv2.resize(frame, (net_input_shape[3], net_input_shape[2]))
        p_frame = p_frame.transpose((2,0,1))
        p_frame = p_frame.reshape(1, *p_frame.shape)

        # Perform inference on the frame
        plugin.async_inference(p_frame)

        # Get the output of inference
        if plugin.wait() == 0:
            result = plugin.extract_output()
            # Draw the output mask onto the input
            out_frame, classes = draw_masks(result, width, height)
            class_names = get_class_names(classes)
            speed = randint(50,70)
            
            ### TODO: Send the class names and speed to the MQTT server
            client.publish("class", json.dumps({"class_names": class_names}))
            client.publish("speedometer", json.dumps({"speed": speed}))
            ### Hint: The UI web server will check for a "class" and
            ### "speedometer" topic. Additionally, it expects "class_names"
            ### and "speed" as the json keys of the data, respectively.
            

        ### TODO: Send frame to the ffmpeg server
        sys.stdout.buffer.write(out_frame)
        sys.stdout.flush()
        

        # Break if escape key pressed
        if key_pressed == 27:
            break

    # Release the capture and destroy any OpenCV windows
    cap.release()
    cv2.destroyAllWindows()
    ### TODO: Disconnect from MQTT
    client.disconnect()


def main():
    args = get_args()
    model = ADAS_MODEL
    infer_on_video(args, model)


if __name__ == "__main__":
    main()


## Solution: Server Communications

[Youtube Video](https://youtu.be/c2cNJgrvHmg)

#### MQTT

In [ ]:
import paho.mqtt.client as mqtt
import socket  # for connection with the MQTT server

In [ ]:
"""
This will set the IP address and port, 
as well as the keep alive interval. 
The keep alive interval is used so that the server 
and client will communicate every 60 seconds 
to confirm their connection is still open, 
if no other communication (such as the inference statistics) 
is received.
"""
HOSTNAME = socket.gethostname()
IPADDRESS = socket.gethostbyname(HOSTNAME)
MQTT_HOST = IPADDRESS
MQTT_PORT = 3001
MQTT_KEEPALIVE_INTERVAL = 60

**Note:** The port here is 3001, instead of the normal MQTT port of 1883, as Udacity's classroom workspace environment only allows ports from 3000-3009 to be used. The real importance is here to make sure this matches to what is set for the MQTT broker server to be listening on, which in this case has also been set to 3001 (we can see this in `config.js` within the MQTT server's files in the workspace).

In [ ]:
# Connecting to the client
client = mqtt.Client()
client.connect(MQTT_HOST, MQTT_PORT, MQTT_KEEPALIVE_INTERVAL)

In [ ]:
# Publishing the statistics to the connected MQTT client
topic = "some_string"
client.publish(topic, json.dumps({"stat_name": statistic}))

The topic here should match to the relevant topic that is being subscribed to from the other end, while the JSON being published should include the relevant name of the statistic for the node server to parse (with the name like the key of a dictionary), with the statistic passed in with it (like the items of a dictionary).

In [ ]:
client.publish("class", json.dumps({"class_names": class_names}))
client.publish("speedometer", json.dumps({"speed": speed}))

In [ ]:
# Disconnecting at the end of processing the input stream
client.disconnect()

#### FFmpeg

FFmpeg does not actually have any real specific imports, although we want the standard `sys` library.

In [ ]:
import sys

This is used as the `ffserver` can be configured to read from `sys.stdout`. Once the output frame has been processed (drawing bounding boxes, semantic masks, etc.), we can write the frame to the stdout buffer and flush it.

In [ ]:
sys.stdout.buffer.write(frame)  
sys.stdout.flush()

And that's it! As long as the MQTT and FFmpeg servers are running and configured appropriately, the information should be able to be received by the final node server, and viewed in the browser.

#### Running the App

To run the app itself, with the UI server, MQTT server, and FFmpeg server also running, we use:

In [ ]:
python app.py | ffmpeg -v warning -f rawvideo -pixel_format bgr24 -video_size 1280x720 -framerate 24 -i - http://0.0.0.0:3004/fac.ffm

This will feed the output of the app to FFmpeg.

## Analyzing Performance Basics

[Youtube Video](https://youtu.be/bGPyVQnLnIw)

We can review:
- [Introduction to the Performance Topics](https://docs.openvinotoolkit.org/2019_R3/_docs_IE_DG_Intro_to_Performance.html)
- [Netflix uses 15% of worldwide bandwidth with its video streaming](https://www.sandvine.com/hubfs/downloads/phenomena/phenomena-presentation-final.pdf)

## Model Use Cases

[Youtube Video](https://youtu.be/rQ0980eYeeA)

We can check on [Deep Learning for Distracted Driving Detection](https://www.nauto.com/blog/nauto-engineering-deep-learning-for-distracted-driver-monitoring)

## Concerning End User Needs

[Youtube Video](https://youtu.be/aHtne0LexrA)

## Recap

[Youtube Video](https://youtu.be/6egpCIvzUl8)

## Lesson Glossary

### OpenCV

A computer vision (CV) library.

### MQTT

A publisher-subscriber protocol. "paho-mqtt" library is a common way of working with MQTT in Python.

### Publish-Subscribe Architecture

### Publisher

### Subscriber

### Topic

### FFmpeg

### Flask

A Python framework useful for web development and another potential option for video streaming to a web browser. [Flask](https://www.fullstackpython.com/flask.html) is used also to build APIs.

### Node Server

A web server built with Node.js that can handle HTTP requests and/or serve up a webpage for viewing in a browser.

## Course Recap

[Youtube Video](https://youtu.be/27C3zdKmt_M)

### Intel® DevMesh

Check out the [Intel® DevMesh](https://devmesh.intel.com/) website for some more awesome projects others have built, join in on existing projects, or even post some of your own!

We can [download the OpenVINO toolkit](https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/choose-download.html).

## Partner with Intel

### Keep growing your Edge AI skills with the resources below:

- Get Started - [Download OpenVINO Today!](https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/choose-download.html?Utm_Medium=Udacity_Platform&utm_source=udacity_fundamentals&utm_content=getstarteddnl&utm_campaign=udacity_2020&cid=certifications&campID=asmo-nar-openvino-da)
- Connect with Experts - [OpenVINO Toolkit Forum](https://software.intel.com/en-us/forums/intel-distribution-of-openvino-toolkit?Utm_Medium=Udacity_Platform&utm_source=udacity_fundamentals&utm_content=connecttlkit&utm_campaign=udacity_2020&cid=certifications&campID=asmo-nar-openvino-da)
- Develop an app using the OpenVINO toolkit - [Access self-paced development guide](https://www.intel.com/content/www/us/en/develop/topics/iot/training/go-to-market-with-openvino.html#linklist_543792205?Utm_Medium=Udacity_Platform&utm_source=udacity_fundamentals&utm_content=devappguide&utm_campaign=udacity_2020&cid=certifications&campID=asmo-nar-openvino-da)
- Get Access to cutting edge AI Hardware with [Intel DevCloud for the Edge](https://software.intel.com/content/www/us/en/develop/tools/devcloud/edge.html?%20utm_medium=Udacity_Platform&utm_source=udacity_fundamentals&utm_content=devcloud&utm_campaign=udacity_2020&cid=certifications&campID=asmo-nar-openvino-da). Develop, test and run your workloads for free on the latest Intel hardware and software.